In [32]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
import wikipedia
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings.openai import OpenAIEmbeddings

In [31]:
api_key = open('api_key.txt').read()
llm = OpenAI(api_key = api_key)

template = """"You are a helpful and compassionate chatbot explaining about mental health disorders to employees in the tech industry at potential risk.
They answered questions on a survey, and have been found to be at risk for {disorder}. 

Context: {disorder_context}

(if the context is irrelevant, ignore it and answer to the best of your abilities)

Information about the person: 
{survey}

Their Question: {question}

Answer: """

prompt = PromptTemplate.from_template(template)

In [39]:
disorder_context = {}

for disorder in ['Anxiety Disorder', 'Mood Disorder']:
    text = ""
    pages = wikipedia.search(disorder)
    for p in pages:
        try: text += wikipedia.page(p).content + '\n\n'
        except: continue
        
    disorder_context[disorder] = text

In [45]:
text_splitter = CharacterTextSplitter(chunk_size=350, 
                                      chunk_overlap=0, separator = '.')
chunked_texts = text_splitter.split_text(disorder_context['Anxiety Disorder'])

db = FAISS.from_texts(chunked_texts, OpenAIEmbeddings(api_key = api_key))
db.save_local('anxiety')

Created a chunk of size 489, which is longer than the specified 350
Created a chunk of size 446, which is longer than the specified 350
Created a chunk of size 368, which is longer than the specified 350
Created a chunk of size 417, which is longer than the specified 350
Created a chunk of size 392, which is longer than the specified 350
Created a chunk of size 1457, which is longer than the specified 350
Created a chunk of size 398, which is longer than the specified 350
Created a chunk of size 689, which is longer than the specified 350
Created a chunk of size 479, which is longer than the specified 350
Created a chunk of size 360, which is longer than the specified 350
Created a chunk of size 451, which is longer than the specified 350
Created a chunk of size 446, which is longer than the specified 350
Created a chunk of size 419, which is longer than the specified 350
Created a chunk of size 591, which is longer than the specified 350
Created a chunk of size 362, which is longer th

In [46]:
# test query
db.similarity_search("I have paranoia", k = 2)

[Document(page_content='These situations can result in an increase in anxiety or a panic attack or feeling the need to separate themselves from family or friends.People with agoraphobia sometimes fear waiting outside for long periods of time; that symptom can be called "macrophobia"'),
 Document(page_content='People with specific phobias understand that their fear is not proportional to the actual potential danger, but they can still become overwhelmed by it.\n\n\n=== Panic disorder ===\n\nWith panic disorder, a person has brief attacks of intense terror and apprehension, often marked by trembling, shaking, confusion, dizziness, or difficulty breathing')]

In [47]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What can I do?"

disorder = "anxiety"
db = FAISS.load_local(f'faiss_databases/{disorder}', OpenAIEmbeddings(api_key = api_key))

context = ""
for t in db.similarity_search(question):
    context += t.page_content + "\n"

ans = llm_chain.invoke({'disorder': disorder, 'disorder_context': context, 'survey': "", 'question': question})['text']
print(ans)

There are several things you can do to help manage your anxiety. One option is to join a self-help or support group, where you can share your experiences with others and learn helpful tools and coping strategies. You can also consider therapy or medication, which can be determined by discussing your specific circumstances and symptoms with a doctor. If one medication does not seem to be effective, there are other options that may be offered to you. It is important to find a treatment plan that works best for you and to continue to communicate with your doctor about your progress. Additionally, practicing self-care and healthy coping mechanisms, such as exercise and mindfulness, can also be beneficial in managing anxiety. Remember to be patient with yourself and seek support from loved ones during this process.
